<a href="https://colab.research.google.com/github/amarie-51/projekt_NLP/blob/main/hate_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [ ]:
import tensorflow as tf


### ** TASK 1**
 Precision, Recall, Balanced F-score and Accuracy.



Przetwarzanie wstępne 

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import re

nltk.download('punkt')



Tworzymy dataframe z treścią tweeta oraz oceną czy jest hejtem

In [ ]:
tags_poprawne=pd.read_csv('/content/test_set_clean_only_tags.txt', header = None)
tweets=open('/content/test_set_clean_only_text.txt')
table_of_tweets=[]
for line in tweets.readlines():
  table_of_tweets.append(line)

tags_poprawne.columns=["Czy hate"]
tweets=pd.DataFrame(table_of_tweets)
tweets.columns=["Tweet"]
df_tweets=pd.concat([tweets,tags_poprawne],axis=1)



In [ ]:
df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x : str.lower(x))
df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x : " ".join(re.findall('[\w]+',x)))

Dodać txt ze stop_words, które trzeba usunąć

Usuwam stop_words

In [ ]:
with open("/content/stop_words.txt", "r") as stop_words:
	lines = stop_words.read().splitlines()


stop_words=lines
def remove_stopWords(s):
    '''For removing stop words
    '''
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x: remove_stopWords(x))


In [ ]:
df_tweets["Tokeny"]=df_tweets["Tweet"]
df_tweets['Tokeny'] = df_tweets.apply(lambda row: nltk.word_tokenize(row['Tweet']), axis=1)
print(df_tweets.columns)


Index(['Tweet', 'Czy hate', 'Tokeny'], dtype='object')


Wizualizacja słów,
 w razie gdyby pojawiły się jeszcze słowa, które warto usunąć np. xd, dodaje do stop_words





In [ ]:
def rozklad_slow(data):
  all_words = ' '.join([text for text in data ])
  wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
  plt.figure(figsize=(10, 7))
  plt.imshow(wordcloud, interpolation="bilinear")
  plt.axis('off')
  plt.show()
rozklad_slow(df_tweets["Tweet"])

sprawdzamy najczestsze słowa w tweetach pozytywnych oraz później w negatywnych

In [ ]:
df_negatywne=df_tweets.loc[df_tweets["Czy hate"]==1].reset_index()

rozklad_slow(df_negatywne['Tweet'])
print(df_negatywne["Tweet"].head)

In [ ]:
df_pozytywne=df_tweets.loc[df_tweets["Czy hate"]==0].reset_index()

rozklad_slow(df_pozytywne["Tweet"])
print(df_pozytywne["Tweet"].head)

Trenowanie sieci neuronowych

regresja liniowa

In [259]:
from sklearn.model_selection import train_test_split
tweets = df_tweets['Tweet'].values
label=df_tweets['Czy hate'].values

tweets_train, tweets_test, label_train, label_test = train_test_split(tweets, label, test_size=0.2, shuffle=True)


In [260]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(tweets_train)

X_train = vectorizer.transform(tweets_train)
X_test  = vectorizer.transform(tweets_test)
print(X_train.shape,X_test.shape)

(800, 3373) (200, 3373)


In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, label_train)
score = classifier.score(X_test, label_test)

print("Accuracy:", score)

# Sieci neuronowe 

In [262]:
import numpy as np
import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
model= Sequential([
                   Dense(64,input_shape=(X_train.shape),activation="relu"),
                   Dropout(0.5),
                   Dense(units=32,activation="relu"),
                   Dropout(0.5),
                   Dense(units=16,activation="relu"),
                   Dense(units=1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])
print(model.summary())

In [ ]:
history= model.fit(X_train,label_train, validation_data=(X_test,label_test), batch_size=8, epochs=10)

pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [220]:
from keras.backend import clear_session
clear_session()

In [ ]:
model.evaluate(X_test,label_test)


In [ ]:
loss, accuracy = model.evaluate(X_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


In [268]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


In [ ]:
plot_history(history) 

Sprawdzamy model

In [ ]:
y_pred=model.predict(X_test).reshape(-1)
print(y_pred[10:20])

y_pred=np.round(y_pred)
print(y_pred[10:20])

In [ ]:
print(label_test[10:20])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(label_test,y_pred))